In [ ]:
!pip install pandas
!pip install pathlib
!pip install yfinance

Defaulting to user installation because normal site-packages is not writeable

You should consider upgrading via the 'c:\program files\python38\python.exe -m pip install --upgrade pip' command.



Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the 'c:\program files\python38\python.exe -m pip install --upgrade pip' command.


Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the 'c:\program files\python38\python.exe -m pip install --upgrade pip' command.


Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the 'c:\program files\python38\python.exe -m pip install --upgrade pip' command.


In [ ]:
import pandas as pd
from pathlib import Path
import yfinance as yf

In [ ]:
def load_dataset (data_filename):
  data_path = Path(data_filename)
  if data_path.exists():
    print(f"Loading {data_path}...")
    data_csv = pd.read_csv(data_path)
    return data_csv
  else:
    print(f"The dataset in {data_path} does not exist")

In [ ]:
#loading partner headlines dataset
data = load_dataset("stock_sentiment_data.csv")
data.rename(columns={'date':'datetime'},inplace = True)
data[['date','time']] = data.datetime.str.split(expand=True)
data.drop(['time'], axis=1, inplace=True)
data.drop(['url'], axis=1, inplace=True)
data.drop(['datetime'], axis=1, inplace=True)
data.drop(data.columns[0], axis=1, inplace=True)
data.head(10)

Loading stock_sentiment_data.csv...


,headline,publisher,stock,sentiment,date
0,Agilent Technologies Announces Pricing of $5……...,GuruFocus,A,2,2020-06-01
1,Agilent (A) Gears Up for Q2 Earnings: What's i...,Zacks,A,2,2020-05-18
2,J.P. Morgan Asset Management Announces Liquida...,GuruFocus,A,1,2020-05-15
3,"Pershing Square Capital Management, L.P. Buys ...",GuruFocus,A,0,2020-05-15
4,Agilent Awards Trilogy Sciences with a Golden ...,GuruFocus,A,0,2020-05-12
5,Agilent Technologies Inc (A) CEO and President...,GuruFocus,A,2,2020-05-11
6,' Stocks Growing Their Earnings Fast,GuruFocus,A,0,2020-05-07
7,Cypress Asset Management Inc Buys Verizon Comm...,GuruFocus,A,0,2020-05-07
8,Hendley & Co Inc Buys American Electric Power ...,GuruFocus,A,0,2020-05-05
9,Teacher Retirement System Of Texas Buys Hologi...,GuruFocus,A,0,2020-05-05


In [ ]:
"""
#get overview of how much data and over what time frame is avaliable
prev_row = data.iloc[-1]
# stock_stats_dict = [stock count, end date, start date]
stock_stats_dict = {}
for index, row in data.iterrows():
  if row["stock"] != prev_row["stock"]:
    stock_stats_dict[row["stock"]] = [1,row["date"],row["date"]]
  if row["stock"] == prev_row["stock"]:
    stock_stats_dict[row["stock"]][0] += 1
    stock_stats_dict[row["stock"]][2] = row["date"]
  prev_row = row
print(stock_stats_dict)
"""

'\n#get overview of how much data and over what time frame is avaliable\nprev_row = data.iloc[-1]\n# stock_stats_dict = [stock count, end date, start date]\nstock_stats_dict = {}\nfor index, row in data.iterrows():\n  if row["stock"] != prev_row["stock"]:\n    stock_stats_dict[row["stock"]] = [1,row["date"],row["date"]]\n  if row["stock"] == prev_row["stock"]:\n    stock_stats_dict[row["stock"]][0] += 1\n    stock_stats_dict[row["stock"]][2] = row["date"]\n  prev_row = row\nprint(stock_stats_dict)\n'

In [ ]:
#get stock price data
"""
stock_price_list = []
for key, values in stock_stats_dict.items():
  print(f"Looking for data for {key}")
  try:
    
    if key == 'MRH':
      pass
    else:
      stock_price = yf.download(key, start=values[2], end=values[1])
      stock_price.reset_index(inplace=True)
      stock_price['Stock'] = key
      stock_price_list.append(stock_price)
  except Exception:
    print(f"There has been an error while looking for data for {key} stock in time period from {values[2]} to {values[1]}")
    pass
stock_price_data = pd.concat(stock_price_list)
print(stock_price_data)
stock_price_data.to_csv("stock_price_data.csv", index=False, header = True, encoding='utf-8-sig')
"""
price_data = load_dataset("stock_price_data.csv")
price_data.head(10)

Loading stock_price_data.csv...


,Date,Open,High,Low,Close,Adj Close,Volume,Stock
0,2012-05-14,27.804007,28.211731,27.496424,27.825464,25.591705,5526574.0,A
1,2012-05-15,29.263233,29.735336,28.905579,28.977110,26.650898,9783064.0,A
2,2012-05-16,29.091558,29.327612,28.397711,28.440628,26.157486,4976321.0,A
3,2012-05-17,29.971388,29.978540,28.340487,28.347639,26.071957,6900248.0,A
4,2012-05-18,28.612303,28.776825,27.625179,27.703863,25.479870,7216057.0,A
5,2012-05-21,27.775393,28.698139,27.753935,28.605150,26.308800,3890215.0,A
6,2012-05-22,28.655222,29.470673,28.340487,28.962805,26.637741,6180139.0,A
7,2012-05-23,28.633762,29.341917,28.175966,29.163090,26.821951,4817788.0,A
8,2012-05-24,29.213161,29.434908,28.934193,29.341917,26.986423,3591182.0,A
9,2012-05-25,29.341917,29.656652,29.177397,29.334764,26.979847,2633552.0,A


In [ ]:
#merging price and sentiment-news dataset
"""
price_data['Headline'] = None
price_data['Publisher'] = None
price_data['Sentiment'] = None
news_added = 0
news_not_added = 0
for index, row in data.iterrows():
  price_index = price_data.loc[(price_data['Stock'] == row['stock']) & (price_data['Date'] == row['date'])].index
  if price_index.empty:
    print(f"Error: Date not avaliable in Price Data")
    news_not_added += 1
  elif row['date'] != price_data.loc[price_index, 'Date'].item():
    print(f"Error: Date Mismatch {row['date']} searched, {price_data[price_index]['Date']} found")
  else:
    price_data.loc[price_index, 'Headline'] = row['headline']
    price_data.loc[price_index, 'Publisher'] = row['publisher']
    price_data.loc[price_index, 'Sentiment'] = row['sentiment']
    news_added += 1
  print(f"{index} / {len(data)} \n{row['stock']}: {row['headline']}")


print(f"Done Merging Datasets! Out of {len(price_data)} Days {news_added} have News. {news_not_added} news not added.")
"""

'\nprice_data[\'Headline\'] = None\nprice_data[\'Publisher\'] = None\nprice_data[\'Sentiment\'] = None\nnews_added = 0\nnews_not_added = 0\nfor index, row in data.iterrows():\n  price_index = price_data.loc[(price_data[\'Stock\'] == row[\'stock\']) & (price_data[\'Date\'] == row[\'date\'])].index\n  if price_index.empty:\n    print(f"Error: Date not avaliable in Price Data")\n    news_not_added += 1\n  elif row[\'date\'] != price_data.loc[price_index, \'Date\'].item():\n    print(f"Error: Date Mismatch {row[\'date\']} searched, {price_data[price_index][\'Date\']} found")\n  else:\n    price_data.loc[price_index, \'Headline\'] = row[\'headline\']\n    price_data.loc[price_index, \'Publisher\'] = row[\'publisher\']\n    price_data.loc[price_index, \'Sentiment\'] = row[\'sentiment\']\n    news_added += 1\n  print(f"{index} / {len(data)} \n{row[\'stock\']}: {row[\'headline\']}")\n\n\nprint(f"Done Merging Datasets! Out of {len(price_data)} Days {news_added} have News. {news_not_added} news 

In [ ]:
# merging price and sentiment datasets using pandas
merged_data = pd.merge(price_data, data, how='left', left_on=['Stock', 'Date'], right_on=['stock', 'date'])
merged_data.drop(['date', 'stock'], axis=1, inplace=True)
merged_data.drop_duplicates(['Date', 'Stock'], keep='last', inplace=True)
merged_data.head(15)
#merged_data.to_csv("stock_price_sentiment_data.csv", index=False, header = True, encoding='utf-8-sig')


,Date,Open,High,Low,Close,Adj Close,Volume,Stock,headline,publisher,sentiment
2,2012-05-14,27.804007,28.211731,27.496424,27.825464,25.591705,5526574.0,A,"Groupon, Agilent: After-Hours Headlines",webmaster,2.0
3,2012-05-15,29.263233,29.735336,28.905579,28.977110,26.650898,9783064.0,A,Agilent Sees Improving Trends - Analyst Blog,Zacks,0.0
5,2012-05-16,29.091558,29.327612,28.397711,28.440628,26.157486,4976321.0,A,"Agilent Still A Name Worth Owning (A, DHR, WAT...",Investopedia,2.0
11,2012-05-17,29.971388,29.978540,28.340487,28.347639,26.071957,6900248.0,A,"Stocks to Watch: Wal-Mart, Sears (Update 1)",TheStreet.Com,2.0
12,2012-05-18,28.612303,28.776825,27.625179,27.703863,25.479870,7216057.0,A,NaN,NaN,NaN
13,2012-05-21,27.775393,28.698139,27.753935,28.605150,26.308800,3890215.0,A,NaN,NaN,NaN
14,2012-05-22,28.655222,29.470673,28.340487,28.962805,26.637741,6180139.0,A,Agilent Pays A Hefty Price For A Fixer-Upper D...,Investopedia,0.0
15,2012-05-23,28.633762,29.341917,28.175966,29.163090,26.821951,4817788.0,A,NaN,NaN,NaN
16,2012-05-24,29.213161,29.434908,28.934193,29.341917,26.986423,3591182.0,A,NaN,NaN,NaN
17,2012-05-25,29.341917,29.656652,29.177397,29.334764,26.979847,2633552.0,A,NaN,NaN,NaN


In [ ]:

percent_news_added = merged_data['headline'].notnull().sum() / len(data) * 100
percent_no_news_days = merged_data['headline'].isnull().sum() / len(merged_data) * 100
print(f"{percent_news_added:.2f} % of news were added. {merged_data['headline'].notnull().sum()} news in total. {100 - percent_no_news_days:.2f}% of days have news")


50.04 % of news were added. 923459 news in total. 12.42% of days have news
